# KELOMPOK 1 (THE DESTROYER )
# MINI PROJECT FITBOT

NAMA ANGGOTA :
1. ADIBA NUHA RABBANI
2. ALLISA OKTAVINA LUKITO WIJAYA
3. ASTINA SALSABILA RANGKUTI
4. ERNA RESTARI
5. MUHAMMAD RAJA REIVAN RIVALDI

In [ ]:
import random # mengimport package random (built-it python)
import json # mengimport package json
import pickle # meimport package pickle
from tabnanny import verbose # mengimport verbose dari tabnanny
import numpy as np # mengimport numpy dan mengaliaskannya sebagai np untuk pembuatan array


import nltk
from nltk.stem import WordNetLemmatizer # mengimport WordNetLemmatizer dari nltk.stem untuk proses lemmatisasi kata-kata
nltk.download('punkt') # mendownload module punkt dari nltk untuk tokenisasi
nltk.download('wordnet') # mendownload module wordnet dari nltk
nltk.download('omw-1.4') # mendownload module omw-1.4 dari nltk

# mengimport package_package dari tensorflow dan keras untuk membuat neural network
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
# codingan ini menampilkan hasil tensorflow model, layer, dan optimizer dengan mengimport sequential, dense, activation, dropout, &SGD.

lemmatizer = WordNetLemmatizer() # menampung function WordNetLemmatizer() ke dalam variabel lemmatizer

intents = json.loads(open('intents.json').read()) # load dan read isi file intens.json

words = [] # variabel berisi list kosong untuk kemudian menampung hasil dari for loop di bawah
classes = [] # variabel berisi list kosong untuk kemudian menampung hasil dari for loop di bawah
documents = [] # variabel berisi list kosong untuk kemudian menampung hasil dari for loop di bawah
ignore_letters = ['?','!',',','.'] # list untuk tanda baca yang diabaikan

for intent in intents['intents']: # for loop untuk setiap item yang ada pada file intent.json
    for pattern in intent['patterns']: # nested for loop untuk setiap pattern yang ada pada intents.json
        word_list = nltk.word_tokenize(pattern) # menampung function untuk tokenisasi pattern ke dalam variabel word_list
        words.extend(word_list) # menambahkan/memperpanjang isi list dalam variabel words dengan item di variabel word_list
        documents.append((word_list, intent['tag'])) # menambahkan variabel documents yang berupa list kosong dengan word_lsi dan tag dalam file intents.json
        if intent['tag'] not in classes: # membuat kondisi jika tag dalam intents.json tidak berada dalam classes
            classes.append(intent['tag']) # menambahkan variabel classes yang berupa list kosong dengan tag yang ada dalam intents.json

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters ] # menampung function untuk melematisasi tiap-tiap kata yang ada pada variabel words syaratnya jika kata-kata tersebut tidak ada pada variabel ignore_letters
words = sorted(set(words)) # merubah kembali menjadi list dan menghilangkan duplikasi

#melakukan short classes 
classes = sorted(set(classes)) # merubah kembali menjadi list dan menghilangkan duplikasi

pickle.dump(words, open('words.pkl', 'wb')) # serialisasi, mengubah isi dari words menjadi words.pkl dalam bentuk byte stream (mode binary)
pickle.dump(classes, open('classes.pkl', 'wb')) # serialisasi,  mengubah isi dari classes menjadi classes.pkl dalam bentuk byte stream (mode binary)

#Membuat Training dan Testing Data Untuk model Train, setiap pola akan diubah dan diinput menjadi angka.
#membuat training data 
training = [] # variabel kosong untuk menampung hasil dari codingan untuk training di bawah
output_empty = [0] * len(classes) # menampung list angka 0 sebanyak panjang item yang ada dalam classes ke dalam variabel output_empty

# pembuatan bag of words untuk persiapan training
for document in documents: # for loop untuk tiap-tiap item pada variabel documets
    bag = [] # membuat variabel bag yang berisikan list kosong
    word_patterns = document[0] # pembuatan variabel word_patterns
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words: #membuat bag of word array dengan 1 
        bag.append(1) if word in word_patterns else bag.append(0) # menambahkan variabel bag yang berisi list dengan angka 1 jika sebuah berada dalam variabel word_patterns jika tidak tambahkan 0 

    output_row = list(output_empty) # copylist dari variabel output_empty untuk variabel output_row
    output_row[classes.index(document[1])] = 1 # menambahkan angka 1 untuk setiap kata yang benar berada dalam suatu dokumen (0 berarti tidak ada kata tertentu dalam dokumen tertentu)
    training.append([bag, output_row])
#shuffle features dan membuat numpy array 
random.shuffle(training)
training = np.array(training) # mengubah variabel training menjadi numpy array
#memisahkan feature dan label untuk training
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# pembuatan neural network
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu' ))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) # compile model

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)# start training, dengan epoch 200 , batch size  5 (satu kali iteraion 5 sampel), verose=1 berarti menampilkan progres bar dan garis dalam setiap epoch
model.save('chatbotmodel.h5', hist) # menyimpan hasil training model
print('selesai!!')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
24/24 [==============================] - 1s 2ms/step - loss: 3.4758 - accuracy: 0.0924 
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 3.3949 - accuracy: 0.0924
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 3.3362 - accuracy: 0.1513
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 3.2295 - accuracy: 0.1597
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 3.0633 - accuracy: 0.2185
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 3.1021 - accuracy: 0.1849
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 2.8600 - accuracy: 0.2605
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 2.7424 - accuracy: 0.2521
Epoch 9/200
24/24 [==============================] - 0s 2ms/step - loss: 2.5300 - accuracy: 0.3277
Epoch 10/200
24/24 [==============================] - 0s 2ms/step - loss: 2.4236 - accuracy: 0.2941
Epoch 11

In [ ]:
import random #perintah untuk mengimport library random
import json #perintah untuk mengimport library json
import pickle #perintah untuk mengimport library pickle
import numpy as np #perintah untuk mengimport library numpy as np
import nltk #perintah untuk mengimport library nltk
from keras.models import load_model # mengimport load_model dari keras.models 
from nltk.stem import WordNetLemmatizer # mengimport package WordNetLemmatizer untuk lemmatisasi kata
#codingan ini menggunakan lemmatizer yang bertujuan untuk menormalisasi pada teks 
lemmatizer = WordNetLemmatizer() # memasukkan function WordNetLemmatizer() ke dalam variabel lemmatizer
intents = json.loads(open("/content/intents.json").read()) # membuka file intents.json (dataset)
words = pickle.load(open('words.pkl', 'rb')) # membuka file words.pkl dalam format binary
classes = pickle.load(open('classes.pkl', 'rb')) # membuka file classes.pkl dalam format binary
model = load_model('chatbotmodel.h5') # load model yang sudah ditraining


# merupakan fungsi yang mengambil input pengguna untuk proses tokenisasi dan lemmatisasi
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)
                      for word in sentence_words]
    return sentence_words

# function yang mengolah input user menjadi bag of words
def bagw(sentence):
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag) #karena membutuhkan fitur nomerik untuk memprediksi kelas
    #maka perintah diatas berfungsi untuk membuat model bag of words untuk teks yang telah di proses sebelumnya
  
def predict_class(sentence): # function untuk memprediksi kelas atau tag dari pertanyaan yang diajukan oleh pengguna
    bow = bagw(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res)
               if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]],
                            'probability': str(r[1])})
        return return_list
  
def get_response(intents_list, intents_json): # function untuk memilih respon acak dari tag yang di prediksi dan mengirimkannya sebagai respons bot
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = ""
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
  
print("Fitbot siap melayani!") # menandakan chatbot siap digunakan

# while loop yang berarti program akan terus berjalan selama user berinteraksi dengan chatbot
while True:
    message = input("") # input untuk user
    ints = predict_class(message) # menerapkan function predict_class dengan parameter message, yang mana message adalah input user (mengolah input user dengan function predict_class)
    res = get_response(ints, intents) # menampilkan respon yang relevan dengan input dari user menggunakan function get_response
    print(res) # menampilkan respon dari chatbot

Chatbot is up!
1/1 [==============================] - 0s 85ms/step
Halo! Saya Fitbot, salam kenal ya! Mau tau tentang diet, kan?
1/1 [==============================] - 0s 18ms/step
Halo! Saya Fitbot, salam kenal ya! Mau tau tentang diet, kan?
1/1 [==============================] - 0s 19ms/step
non-exercise activity thermogenesis (NEAT) adalah semua aktifitas fisik yang kamu lakukan setiap hari yang tidak dimaksudkan sebagai bentuk olahraga, misalnya berjalan kaki ke tempat tujuan, berdiri, naik tangga, mengerjakan pekerjaan rumah, berkebun, bermain bersama anak, dan lain-lain.
1/1 [==============================] - 0s 20ms/step
diet tidak perlu bermodalkan uang yang banyak, asal kamu tahu caranya untuk memilih menu dan bahan makanan yang murah. seperti contohnya dada ayam, buah, dan sayuran masih terbilang terjangkau harganya setelah itu barulah siasati bagaimana cara mengolahnya agar sehat, enak, dan rendah kalori
1/1 [==============================] - 0s 17ms/step
puasa sendiri memil